<a href="https://colab.research.google.com/github/eissa2002/Customer-Service-chatbot/blob/main/Customer_service_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# import necessary libraries
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json



In [24]:
# Load the intents file
with open('/content/intents (2) (1) (2).json') as file:
    data = json.load(file)

In [25]:
# Preprocess the data
stemmer = LancasterStemmer()

words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize the text and stem each word
        wrds = nltk.word_tokenize(pattern)
        wrds = [stemmer.stem(w) for w in wrds]
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])

    # Add the label to the labels list
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [26]:
# Remove duplicates and sort the words
words = sorted(list(set(words)))

In [27]:
# Label encoding
training = []
output = []
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    for w in words:
        if w in doc:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

In [28]:
# Building the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, input_shape=(len(training[0]),)),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(len(output[0]), activation='softmax'),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [29]:
# Training the model
model.fit(training, output, epochs=300, batch_size=8)

Train on 106 samples
Epoch 1/300
106/106 [==============================] - 0s 752us/sample - loss: 3.3618 - acc: 0.0566
Epoch 2/300
106/106 [==============================] - 0s 228us/sample - loss: 3.3348 - acc: 0.0566
Epoch 3/300
106/106 [==============================] - 0s 244us/sample - loss: 3.3109 - acc: 0.0755
Epoch 4/300
106/106 [==============================] - 0s 218us/sample - loss: 3.2894 - acc: 0.0755
Epoch 5/300
106/106 [==============================] - 0s 225us/sample - loss: 3.2664 - acc: 0.0755
Epoch 6/300
106/106 [==============================] - 0s 253us/sample - loss: 3.2433 - acc: 0.0943
Epoch 7/300
106/106 [==============================] - 0s 260us/sample - loss: 3.2197 - acc: 0.1132
Epoch 8/300
106/106 [==============================] - 0s 263us/sample - loss: 3.1956 - acc: 0.1226
Epoch 9/300
106/106 [==============================] - 0s 226us/sample - loss: 3.1688 - acc: 0.1415
Epoch 10/300
106/106 [==============================] - 0s 449us/sample - loss:

In [ ]:
# Building the chatbot
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return np.array(bag)
def get_response(user_input):
    bag = bag_of_words(user_input, words)
    bag = np.reshape(bag, (1, len(bag)))
    results = model.predict(bag)
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data['intents']:
        if tg['tag'] == tag:
            responses = tg['responses']

    return random.choice(responses)



def chat():
    print("Start talking with the bot (type 'quit' to exit)")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break

        response = get_response(user_input)
        print("zed:", response)
if __name__ == '__main__':
    chat()

Start talking with the bot (type 'quit' to exit)
